In [1]:
from typing import Dict

import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

from lonboard.colormap import apply_continuous_cmap
from lonboard import Map, ScatterplotLayer
from palettable.cartocolors.sequential import BurgYl_2
from geojson_pydantic import Feature, Polygon

In [2]:
BASE_URL = "http://localhost:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [3]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['sum_pop_2020', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020', 'sum_pop_m_2020', 'sum_pop_f_2020']


In [4]:
AOIModel = Feature[Polygon, Dict]

# kenya
aoi = {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [41.85508309264397, -1.68325],
            [40.98105, -2.49979],
            [40.993, -3.444],
            [41.58513, -3.91909],
            [40.88477, -4.95913],
            [39.55938425876585, -4.437641590288629],
            [39.25451, -3.42206],
            [37.7669, -3.67712],
            [37.69869, -3.09699],
            [34.07262, -1.05982],
            [33.90371119710453, -0.95],
            [33.893568969666944, 0.109813537861896],
            [34.18, 0.515],
            [34.6721, 1.17694],
            [35.03599, 1.90584],
            [34.59607, 3.05374],
            [34.47913, 3.5556],
            [35.298007118232946, 4.77696566346189],
            [35.817447662353516, 4.77696566346189],
            [36.159078632855646, 4.447864127672769],
            [36.85509323800812, 4.447864127672769],
            [38.120915, 3.598605],
            [38.43697, 3.58851],
            [38.67114, 3.61607],
            [38.89251, 3.50074],
            [39.55938425876585, 3.42206],
            [39.85494, 3.83879],
            [40.76848, 4.25702],
            [41.1718, 3.91909],
            [41.85508309264397, 2.97959],
            [41.58513, 2.09],
            [40.993, 1.657],
            [40.98105, 1.002],
            [41.85508309264397, -1.68325]
          ]
        ]
      },
      "properties": {
        "name": "Kenya"
      }
    }


feat = AOIModel(**aoi)

In [6]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": ["sum_pop_2020", "sum_pop_f_2020", "sum_pop_m_2020"], 
    "geometry": "point"
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)

In [7]:
df['geometry'] = df['geometry'].apply(lambda geom: shape(geom))
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf

,hex_id,geometry,sum_pop_2020,sum_pop_f_2020,sum_pop_m_2020
0,866a4a48fffffff,POINT (36.98919 -1.94819),399.860905,189.675539,210.185366
1,866a4a497ffffff,POINT (35.56927 4.73105),582.555159,276.337255,306.217904
2,866a4a49fffffff,POINT (40.38384 3.66753),749.911237,355.723245,394.187992
3,866a4a4d7ffffff,POINT (40.40089 -3.11104),863.888290,418.309236,445.579054
4,866a5820fffffff,POINT (36.83518 2.89708),525.085147,249.076134,276.009012
...,...,...,...,...,...
16212,867b5dd77ffffff,POINT (38.79038 -1.26091),-36.000000,-18.000000,-18.000000
16213,867b5dd87ffffff,POINT (35.60388 0.82106),-36.000000,-18.000000,-18.000000
16214,867b5dd8fffffff,POINT (36.47198 0.42433),-36.000000,-18.000000,-18.000000
16215,867b5dd9fffffff,POINT (39.97969 0.30059),-36.000000,-18.000000,-18.000000


In [8]:
max = gdf["sum_pop_2020"].max()
min = gdf["sum_pop_2020"].min()
normalized_sum_pop_2020 = (gdf["sum_pop_2020"] - min) / (max - min)
normalized_sum_pop_2020

layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=apply_continuous_cmap(normalized_sum_pop_2020, BurgYl_2, alpha=0.7))
m = Map(layer)
m


Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x155a2c8e0>
[
  [
    2…